<a href="https://colab.research.google.com/github/Rahulraj31/ML-Language-Translator/blob/main/ML_Language_Translator_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Leanirng Language Translator 

**Dataset Link- http://www.manythings.org/anki/fra-eng.zip**

In [1]:
from tensorflow.keras.models import  Model
from tensorflow.keras.layers import Input, LSTM,Dense 
import numpy as np

In [2]:
"""from zipfile import ZipFile
  
file_name = "/content/drive/MyDrive/fra-eng.zip"
  
with ZipFile(file_name, 'r') as zip:
    zip.extractall("/content")"""
   

'from zipfile import ZipFile\n  \nfile_name = "/content/drive/MyDrive/fra-eng.zip"\n  \nwith ZipFile(file_name, \'r\') as zip:\n    zip.extractall("/content")'

In [3]:
batch_size=64   #batch size for training
epochs =100     # no. of epochs  
latent_dim=256    #dimension of the encoding space
num_samples =10000    
data_path="/content/fra.txt"

In [4]:
# Vectorize Data 

input_texts =[]
target_texts =[]

input_characters = set()
target_characters = set()

with open(data_path,'r',encoding='utf-8') as f:
  lines=f.read().split('\n')

for line in lines[: min(num_samples,len(lines)-1)]:  
  input_text,target_text, _ = line.split('\t')

  # We use "tab" as the start sequence character for the targets and "\n" as end sequence  character
  target_text= '\t' + target_text + '\n'
  input_texts.append(input_text)
  target_texts.append(target_text)

  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)

  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)     


In [5]:
input_characters= sorted(list(input_characters))
target_characters=sorted(list(target_characters))
num_encoder_tokens=len(input_characters)
num_decoder_tokens=len(target_characters)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length =max([len(txt) for txt in target_texts])

In [6]:
print("Numbers of samples:",len(input_texts))
print("Number of unique input tokens",num_encoder_tokens)
print("Number of unique target tokens",num_decoder_tokens)
print("Max encoder seq length",max_encoder_seq_length)
print("Max decoder seq length",max_decoder_seq_length)

Numbers of samples: 10000
Number of unique input tokens 71
Number of unique target tokens 92
Max encoder seq length 15
Max decoder seq length 59


In [7]:
input_characters[:20]

[' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '5',
 '7',
 '8',
 '9',
 ':',
 '?']

In [8]:
target_texts[:10]

['\tVa !\n',
 '\tMarche.\n',
 '\tBouge !\n',
 '\tSalut !\n',
 '\tSalut.\n',
 '\tCours\u202f!\n',
 '\tCourez\u202f!\n',
 '\tPrenez vos jambes à vos cous !\n',
 '\tFile !\n',
 '\tFilez !\n']

In [9]:
target_characters[:20]

['\t',
 '\n',
 ' ',
 '!',
 '%',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '5',
 '8',
 '9',
 ':']

In [10]:
input_token_index =dict([(char,i) for i,char in enumerate(input_characters)])
target_token_index =dict([(char,i) for i,char in enumerate(target_characters)])

In [11]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '5': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 ':': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'Q': 36,
 'R': 37,
 'S': 38,
 'T': 39,
 'U': 40,
 'V': 41,
 'W': 42,
 'Y': 43,
 'a': 44,
 'b': 45,
 'c': 46,
 'd': 47,
 'e': 48,
 'f': 49,
 'g': 50,
 'h': 51,
 'i': 52,
 'j': 53,
 'k': 54,
 'l': 55,
 'm': 56,
 'n': 57,
 'o': 58,
 'p': 59,
 'q': 60,
 'r': 61,
 's': 62,
 't': 63,
 'u': 64,
 'v': 65,
 'w': 66,
 'x': 67,
 'y': 68,
 'z': 69,
 'é': 70}

In [12]:
target_token_index

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 '(': 7,
 ')': 8,
 ',': 9,
 '-': 10,
 '.': 11,
 '0': 12,
 '1': 13,
 '2': 14,
 '3': 15,
 '5': 16,
 '8': 17,
 '9': 18,
 ':': 19,
 '?': 20,
 'A': 21,
 'B': 22,
 'C': 23,
 'D': 24,
 'E': 25,
 'F': 26,
 'G': 27,
 'H': 28,
 'I': 29,
 'J': 30,
 'K': 31,
 'L': 32,
 'M': 33,
 'N': 34,
 'O': 35,
 'P': 36,
 'Q': 37,
 'R': 38,
 'S': 39,
 'T': 40,
 'U': 41,
 'V': 42,
 'Y': 43,
 'a': 44,
 'b': 45,
 'c': 46,
 'd': 47,
 'e': 48,
 'f': 49,
 'g': 50,
 'h': 51,
 'i': 52,
 'j': 53,
 'k': 54,
 'l': 55,
 'm': 56,
 'n': 57,
 'o': 58,
 'p': 59,
 'q': 60,
 'r': 61,
 's': 62,
 't': 63,
 'u': 64,
 'v': 65,
 'w': 66,
 'x': 67,
 'y': 68,
 'z': 69,
 '\xa0': 70,
 '«': 71,
 '»': 72,
 'À': 73,
 'Ç': 74,
 'É': 75,
 'Ê': 76,
 'à': 77,
 'â': 78,
 'ç': 79,
 'è': 80,
 'é': 81,
 'ê': 82,
 'î': 83,
 'ï': 84,
 'ô': 85,
 'ù': 86,
 'û': 87,
 'œ': 88,
 '\u2009': 89,
 '’': 90,
 '\u202f': 91}

In [13]:
# Preparation for one hot encoding
encoder_input_data=np.zeros(
    (len(input_texts),max_encoder_seq_length,num_encoder_tokens),dtype='float32'
)

decoder_input_data=np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32'
)

decoder_target_data=np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32'
)

In [14]:
# Performing One hot encoding
for i ,(input_text,target_text) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_text):
      encoder_input_data[i,t,input_token_index[char]] = 1.
  encoder_input_data[i,t+1:,input_token_index[' ']] =1.
  for t, char in enumerate(target_text):
    #decoder_target_data is ahead of decoder_input_data by one timesetep
      decoder_input_data[i,t,target_token_index[char]] =1.
      if t >0:
      # decoder target data will be ahead by one timestep 
      #and will not include the start character
          decoder_target_data[i,t-1,target_token_index[char]] = 1.

  decoder_input_data[i,t+1:,target_token_index[' ']] = 1.
  decoder_target_data[i,t:,target_token_index[' ']] = 1.


In [15]:
encoder_input_data[0].shape,decoder_input_data[0].shape

((15, 71), (59, 92))

In [16]:
# Define input Sequence 

encoder_inputs = Input(shape=(None,num_encoder_tokens))
encoder=LSTM(latent_dim,return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard encoder outputs and only keep state 
encoder_states = [state_h,state_c]

In [17]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)



In [18]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2,verbose=1)

Epoch 1/100
125/125 [==============================] - 45s 340ms/step - loss: 1.1347 - accuracy: 0.7371 - val_loss: 1.0248 - val_accuracy: 0.7108
Epoch 2/100
125/125 [==============================] - 42s 334ms/step - loss: 0.8093 - accuracy: 0.7803 - val_loss: 0.8275 - val_accuracy: 0.7718
Epoch 3/100
125/125 [==============================] - 42s 334ms/step - loss: 0.6546 - accuracy: 0.8171 - val_loss: 0.6977 - val_accuracy: 0.7971
Epoch 4/100
125/125 [==============================] - 41s 332ms/step - loss: 0.5714 - accuracy: 0.8338 - val_loss: 0.6312 - val_accuracy: 0.8159
Epoch 5/100
125/125 [==============================] - 41s 331ms/step - loss: 0.5251 - accuracy: 0.8456 - val_loss: 0.5918 - val_accuracy: 0.8269
Epoch 6/100
125/125 [==============================] - 41s 330ms/step - loss: 0.4886 - accuracy: 0.8560 - val_loss: 0.5656 - val_accuracy: 0.8343
Epoch 7/100
125/125 [==============================] - 41s 329ms/step - loss: 0.4602 - accuracy: 0.8644 - val_loss: 0.5397 -

**To decode a test sentence, we will repeatedly:**

1) Encode the input sentence and retrieve the initial decoder state

2) Run one step of the decoder with this initial state and a "start of sequence" token as target. The output will be the next target character.

3) Append the target character predicted and repeat.

In [22]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# reverse-Lookup token index to decode sequences back to something readable

reverse_input_char_index = dict ([
    (i,char) for i,char in enumerate(input_token_index)])
reverse_target_char_index = dict ([
    (i,char) for i,char in enumerate(target_token_index)]
)



def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence




for seq_index in range(10):
  #take one sequence for training and trying out decoding
  input_seq  = encoder_input_data[seq_index:seq_index+1]
  decoded_sentence= decode_sequence(input_seq)

  print("Input Sequence",input_texts[seq_index])
  print("decoded Sequence",decoded_sentence)
  
  print("-----")    

Input Sequence Go.
decoded Sequence Va !

-----
Input Sequence Go.
decoded Sequence Va !

-----
Input Sequence Go.
decoded Sequence Va !

-----
Input Sequence Hi.
decoded Sequence Salut !

-----
Input Sequence Hi.
decoded Sequence Salut !

-----
Input Sequence Run!
decoded Sequence Cours !

-----
Input Sequence Run!
decoded Sequence Cours !

-----
Input Sequence Run!
decoded Sequence Cours !

-----
Input Sequence Run!
decoded Sequence Cours !

-----
Input Sequence Run!
decoded Sequence Cours !

-----
